In [13]:
import os

import torch
import torch.nn.functional as F

import torch_geometric
from torch_geometric.nn import SAGEConv
from torch_geometric.datasets import Planetoid
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

1.11.0


In [3]:
use_cuda_if_available = False
use_cuda_if_available = True

# Load the dataset

Cora dataset is about paper citations

In [5]:
dataset = Planetoid(root="cora",name= "Cora")

Processing...
Done!


##### Dataset properties

In [6]:
print(dataset)
print("number of graphs:\t\t",len(dataset))
print("number of classes:\t\t",dataset.num_classes)
print("number of node features:\t",dataset.num_node_features)
print("number of edge features:\t",dataset.num_edge_features)

Cora()
number of graphs:		 1
number of classes:		 7
number of node features:	 1433
number of edge features:	 0


##### Dataset shapes

In [7]:
print(dataset.data)

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


* `x` has 2,708 nodes with 1,433 features
* `y` has 2,708 entries (a label for each node)
* `edge_index` is an array of shape (2, `n`), where:
  * `n` is the number of edges
  * 2 is always fixed and represents index of source node and index of target node
* `train/val/test_mask` are boolean masks that give the indices of train, validation and test data points 

Here's how the data looks like

In [10]:
print("edge_index:\t\t",dataset.data.edge_index.shape)
print(dataset.data.edge_index)
print("\n")
print("train_mask:\t\t",dataset.data.train_mask.shape)
print(dataset.data.train_mask)
print("\n")
print("x:\t\t",dataset.data.x.shape)
print(dataset.data.x)
print("\n")
print("y:\t\t",dataset.data.y.shape)
print(dataset.data.y)

edge_index:		 torch.Size([2, 10556])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])


train_mask:		 torch.Size([2708])
tensor([ True,  True,  True,  ..., False, False, False])


x:		 torch.Size([2708, 1433])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


y:		 torch.Size([2708])
tensor([3, 4, 4,  ..., 3, 3, 3])


We only have a single graph so we can take it from the dataset

In [14]:
data = dataset[0]

Build a simple network with SAGEConv

In [18]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = SAGEConv(
            in_channels=data.num_features,
            out_channels=data.num_classes,
            aggr="max") # max, mean, add ...)

    def forward(self):
        x = self.conv(data.x, data.edge_index)
        out = F.log_softmax(x, dim=1)
        return out

Define model, and send data

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() and use_cuda_if_available else 'cpu')
model, data = Net().to(device), data.to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [22]:
# Train step
def train():
    model.train()
    optimizer.zero_grad()
    F.nll_loss(model()[data.train_mask], data.y[data.train_mask]).backward()
    optimizer.step()


def test():
    model.eval()
    logits, accs = model(), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs

Simple learning loop

In [25]:
best_val_acc = test_acc = 0

for epoch in range(1,10):
    train()
    _, val_acc, tmp_test_acc = test()
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        test_acc = tmp_test_acc
    log = 'Epoch: {:03d}, Val: {:.4f}, Test: {:.4f}'
    
    print(log.format(epoch, best_val_acc, test_acc))

Epoch: 001, Val: 0.7240, Test: 0.7280
Epoch: 002, Val: 0.7280, Test: 0.7280
Epoch: 003, Val: 0.7280, Test: 0.7280
Epoch: 004, Val: 0.7280, Test: 0.7280
Epoch: 005, Val: 0.7280, Test: 0.7280
Epoch: 006, Val: 0.7280, Test: 0.7280
Epoch: 007, Val: 0.7280, Test: 0.7280
Epoch: 008, Val: 0.7280, Test: 0.7280
Epoch: 009, Val: 0.7280, Test: 0.7280
